<a href="https://colab.research.google.com/github/camilla-scandola/project-shark-attack/blob/main/quest_sharkatack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pycountry

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import re

In [6]:
df = pd.read_excel('GSAF5.xls')

In [7]:
pd.set_option('display.max_columns', None)
df.head()


,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Time,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
0,16th August 2025,2025.0,Provoked,USA,Florida,Cayo Costa Boca Grande,Fishing,Shawn Meuse,M,?,Laceration to right leg below the knee,N,1055 hrs,Lemon shark 1.8 m (6ft),Johannes Marchand: Kevin McMurray Trackingshar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18th August,2025.0,Unprovoked,Australia,NSW,Cabarita Beach,Surfing,Brad Ross,M,?,None sustained board severly damaged,N,0730hrs,5m (16.5ft) Great White,Bob Myatt GSAF The Guardian: 9 News: ABS News:...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17th August,2025.0,Unprovoked,Bahamas,Atlantic Ocean near Big Grand Cay,North of Grand Bahama near Freeport,Spearfishing,Not stated,M,63,Severe injuries no detail,N,1300hrs,Undetermined,Ralph Collier GSAF and Kevin MCMurray Tracking...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7th August,2025.0,Unprovoked,Australia,NSW,Tathra Beach,Surfing,Bowie Daley,M,9,None sustained board severely damaged,N,1630hrs,Suspected Great White,Bob Myatt GSAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1st August,2025.0,Unprovoked,Puerto Rico,Carolina,Carolina Beach,Wading,Eleonora Boi,F,39,Bite to thigh area,N,Not stated,Undetermined,Kevin McMurray Trackingsharks.com: NY Post,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#overview of the dataset

missing_country = int(df['Country'].isna().sum())
print(missing_country)

missing_state = int(df['State'].isna().sum())
print(missing_state)

missing_location = int(df['Location'].isna().sum())
print(missing_location)

print(df['Country'].tail())

#suggestion: drop all the null values under country (because they won't make a big difference in the analysis, being 50 out of 7041)
#df['Country'].dropna()

50
485
567
7037             AUSTRALIA
7038             AUSTRALIA
7039                   USA
7040                PANAMA
7041    CEYLON (SRI LANKA)
Name: Country, dtype: object


In [9]:
#suggestion: the number of missing values in states and locations is a lot more significant, so I'd replace them with undefined for the time being
df['State'].fillna('undefined')
df['Location'].fillna('undefined')

0                    Cayo Costa Boca Grande
1                            Cabarita Beach
2       North of Grand Bahama near Freeport
3                             Tathra Beach 
4                            Carolina Beach
                       ...                 
7037                            Roebuck Bay
7038                              undefined
7039                         Ocracoke Inlet
7040                   Panama Bay 8ºN, 79ºW
7041    Below the English fort, Trincomalee
Name: Location, Length: 7042, dtype: object

In [10]:
#if needed, imported pycountry to check all standard country names + 

import pycountry

df['Country_cleaned'] = df['Country'].astype(str).str.strip().str.lower().str.title()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    
    print(df['Country_cleaned'].value_counts())

Country_cleaned
Usa                                      2570
Australia                                1507
South Africa                              599
New Zealand                               146
Bahamas                                   141
Papua New Guinea                          136
Brazil                                    122
Mexico                                    107
Italy                                      72
Fiji                                       70
New Caledonia                              66
Philippines                                65
Reunion                                    60
Egypt                                      53
Mozambique                                 50
Nan                                        50
Cuba                                       49
Spain                                      47
India                                      41
French Polynesia                           39
Japan                                      36
Croatia           

In [11]:
# Mapping of common mismatches to ISO pycountry names
replacements = {
    # Abbreviations & Short forms
    "Usa": "United States",
    "UK": "United Kingdom",
    "UAE": "United Arab Emirates",

    # Alternative spellings
    "South Korea": "Korea, Republic of",
    "North Korea": "Korea, Democratic People's Republic of",
    "Russia": "Russian Federation",
    "Iran": "Iran, Islamic Republic of",
    "Syria": "Syrian Arab Republic",
    "Tanzania": "Tanzania, United Republic of",
    "Vietnam": "Viet Nam",
    "Turkey": "Türkiye",
    "Bolivia": "Bolivia, Plurinational State of",
    "Venezuela": "Venezuela, Bolivarian Republic of",
    "Moldova": "Moldova, Republic of",
    "Laos": "Lao People's Democratic Republic",
    "Palestinian Territories": "Palestine, State of",
    "Micronesia": "Micronesia, Federated States of",
    "Macedonia": "North Macedonia",

    # Historical names
    "Burma": "Myanmar",
    "Ceylon": "Sri Lanka",
    "Ceylon (Sri Lanka)": "Sri Lanka",
    "Western Samoa": "Samoa",
    "Zaire": "Congo, The Democratic Republic of the",

    # Variations / territories that map to countries
    "Curacao": "Curaçao",
    "St. Martin": "Saint Martin (French part)",
    "St Martin": "Saint Martin (French part)",
    "St. Maartin": "Sint Maarten (Dutch part)",
    "Reunion": "Réunion",
    "Reunion Island": "Réunion",
    "Bahrein": "Bahrain",
    "Cape Verde": "Cabo Verde",
    "Taiwan": "Taiwan, Province of China",
    "Maldive Islands": "Maldives",
}

df["Country_cleaned"] = df["Country_cleaned"].replace(replacements)

df.head()

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Time,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22,Country_cleaned
0,16th August 2025,2025.0,Provoked,USA,Florida,Cayo Costa Boca Grande,Fishing,Shawn Meuse,M,?,Laceration to right leg below the knee,N,1055 hrs,Lemon shark 1.8 m (6ft),Johannes Marchand: Kevin McMurray Trackingshar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States
1,18th August,2025.0,Unprovoked,Australia,NSW,Cabarita Beach,Surfing,Brad Ross,M,?,None sustained board severly damaged,N,0730hrs,5m (16.5ft) Great White,Bob Myatt GSAF The Guardian: 9 News: ABS News:...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Australia
2,17th August,2025.0,Unprovoked,Bahamas,Atlantic Ocean near Big Grand Cay,North of Grand Bahama near Freeport,Spearfishing,Not stated,M,63,Severe injuries no detail,N,1300hrs,Undetermined,Ralph Collier GSAF and Kevin MCMurray Tracking...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bahamas
3,7th August,2025.0,Unprovoked,Australia,NSW,Tathra Beach,Surfing,Bowie Daley,M,9,None sustained board severely damaged,N,1630hrs,Suspected Great White,Bob Myatt GSAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Australia
4,1st August,2025.0,Unprovoked,Puerto Rico,Carolina,Carolina Beach,Wading,Eleonora Boi,F,39,Bite to thigh area,N,Not stated,Undetermined,Kevin McMurray Trackingsharks.com: NY Post,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Puerto Rico


In [12]:
pycountry_names = {c.name for c in pycountry.countries}
df = df[df['Country_cleaned'].isin(pycountry_names)]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df['Country_cleaned'].value_counts())

Country_cleaned
United States                        2570
Australia                            1507
South Africa                          599
New Zealand                           146
Bahamas                               141
Papua New Guinea                      136
Brazil                                122
Mexico                                107
Italy                                  72
Fiji                                   70
New Caledonia                          66
Réunion                                65
Philippines                            65
Egypt                                  53
Mozambique                             50
Cuba                                   49
Spain                                  47
India                                  41
French Polynesia                       39
Japan                                  36
Croatia                                35
Jamaica                                32
Panama                                 32
Solomon Islands   

In [13]:
#Columns Fatal + Deaths
#checking all unique values in Fatal, to understand how I can better clean it
print(df['Fatal Y/N'].unique())

['N' 'Y' 'F' 'M' nan 'n' 'Nq' 'UNKNOWN' 2017 'Y x 2' ' N' 'N ']


In [14]:
#cleaning Fatal and printing unique values
df['Fatal Y/N cleaned'] = df['Fatal Y/N'].astype(str).str.strip().str.upper().apply(lambda x: re.sub(r'[^A-Z]', '', x))
print(df['Fatal Y/N cleaned'].unique())

#add column "Death" and assign a numeric value to Fatal, yes or no
deaths_count = {'N': 0, 'Y': 1}
df['Death'] = df['Fatal Y/N cleaned'].map(deaths_count)
deaths_count = int(df['Death'].sum())
print(deaths_count)

['N' 'Y' 'F' 'M' 'NAN' 'NQ' 'UNKNOWN' '' 'YX']
1376


In [15]:
#checking all unique values in Fatal after clean-up
counts = df['Fatal Y/N cleaned'].value_counts(dropna=False)
print(counts)

#all data that is neither Y nor N, is replaced with NaN in the transition from numeric values
counts = df['Death'].value_counts(dropna=False)
print(counts)

Fatal Y/N cleaned
N          4766
Y          1376
NAN         535
UNKNOWN      69
F             5
M             3
NQ            1
              1
YX            1
Name: count, dtype: int64
Death
0.0    4766
1.0    1376
NaN     615
Name: count, dtype: int64


In [16]:
df.head()

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Time,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22,Country_cleaned,Fatal Y/N cleaned,Death
0,16th August 2025,2025.0,Provoked,USA,Florida,Cayo Costa Boca Grande,Fishing,Shawn Meuse,M,?,Laceration to right leg below the knee,N,1055 hrs,Lemon shark 1.8 m (6ft),Johannes Marchand: Kevin McMurray Trackingshar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,N,0.0
1,18th August,2025.0,Unprovoked,Australia,NSW,Cabarita Beach,Surfing,Brad Ross,M,?,None sustained board severly damaged,N,0730hrs,5m (16.5ft) Great White,Bob Myatt GSAF The Guardian: 9 News: ABS News:...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Australia,N,0.0
2,17th August,2025.0,Unprovoked,Bahamas,Atlantic Ocean near Big Grand Cay,North of Grand Bahama near Freeport,Spearfishing,Not stated,M,63,Severe injuries no detail,N,1300hrs,Undetermined,Ralph Collier GSAF and Kevin MCMurray Tracking...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bahamas,N,0.0
3,7th August,2025.0,Unprovoked,Australia,NSW,Tathra Beach,Surfing,Bowie Daley,M,9,None sustained board severely damaged,N,1630hrs,Suspected Great White,Bob Myatt GSAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Australia,N,0.0
4,1st August,2025.0,Unprovoked,Puerto Rico,Carolina,Carolina Beach,Wading,Eleonora Boi,F,39,Bite to thigh area,N,Not stated,Undetermined,Kevin McMurray Trackingsharks.com: NY Post,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Puerto Rico,N,0.0


# Create a new boolean column called ‘Surfing’ 

In this column only surfing-related activities are represented.

In [17]:
# Define search terms related to water sports activities
search_terms = ['surf', 'surfing', 'bodyboard', 'bodyboarding', 'longboard', 'Longboarding', 'windsurf', 'windsurfing', 'kite', 'paddle']

# Create a new boolean column 'Surfing' in the dataframe
# This column will be True if any of the search terms are found in the 'Activity' column
# The '|'.join() creates a regex pattern that matches any of the search terms
# case=False makes the search case-insensitive
# na=False treats NaN values as empty strings instead of raising an error
df['Surfing'] = df['Activity'].str.contains('|'.join(search_terms), case=False, na=False)

# Relocate 'Surfing' column at the fourth position (index 7) 
cols = list(df.columns)
cols.remove('Surfing')
cols.insert(7, 'Surfing')
df = df[cols]

# Cleaning the 'Year' Column

In this section, we'll handle the missing and inconsistent values in the 'Year' column to ensure the data is in the correct format for analysis.

In [18]:
# Display information about the 'Year' column in the dataframe
df['Year'].info()

<class 'pandas.core.series.Series'>
Index: 6757 entries, 0 to 7041
Series name: Year
Non-Null Count  Dtype  
--------------  -----  
6756 non-null   float64
dtypes: float64(1)
memory usage: 105.6 KB


In [19]:
# Inspect the null values in the Year column
df[df['Year'].isnull()]

,Date,Year,Type,Country,State,Location,Activity,Surfing,Name,Sex,Age,Injury,Fatal Y/N,Time,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22,Country_cleaned,Fatal Y/N cleaned,Death
896,Reported 08-Jan-2017,NaN,Invalid,AUSTRALIA,Queensland,NaN,Spearfishing,False,Kerry Daniel,M,35,"No attack, shark made a threat display",NaN,NaN,Bull shark,Liquid Vision 1/8/2017,2017.01.08.R-KerryDaniel.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.01.08.R,2017.01.08.R,6142.0,NaN,NaN,Australia,NAN,NaN


In [20]:
# Fill the two null entries in the 'Year' column by importing the year from the 'Date' column

import re

# Extract the year (4 digits) from the 'Date' column for row 896
year_896 = re.search(r'\d{4}', df.loc[896, 'Date']).group()

# Update the 'Year' column with the extracted year for row 896
df.loc[896, 'Year'] = float(year_896)

df.loc[[896]]

,Date,Year,Type,Country,State,Location,Activity,Surfing,Name,Sex,Age,Injury,Fatal Y/N,Time,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22,Country_cleaned,Fatal Y/N cleaned,Death
896,Reported 08-Jan-2017,2017.0,Invalid,AUSTRALIA,Queensland,NaN,Spearfishing,False,Kerry Daniel,M,35,"No attack, shark made a threat display",NaN,NaN,Bull shark,Liquid Vision 1/8/2017,2017.01.08.R-KerryDaniel.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.01.08.R,2017.01.08.R,6142.0,NaN,NaN,Australia,NAN,NaN


In [21]:
# Convert the 'Year' column to integer data type for numerical operations
df['Year'] = df['Year'].astype(int)

# Print the data type to verify the conversion was successful
print(df['Year'].dtype)

int64


In [22]:
# Get all unique values from the 'Year' column in the dataframe
# Quick overview to check if there is any recent missing year
df['Year'].unique()

array([2025, 2024, 2026, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016,
       2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005,
       2004, 2003, 2002, 2001, 2000, 1999, 1998, 1997, 1996, 1995, 1984,
       1994, 1993, 1992, 1991, 1990, 1989, 1969, 1988, 1987, 1986, 1985,
       1983, 1982, 1981, 1980, 1979, 1978, 1977, 1976, 1975, 1974, 1973,
       1972, 1971, 1970, 1968, 1967, 1966, 1965, 1964, 1963, 1962, 1961,
       1960, 1959, 1958, 1957, 1956, 1955, 1954, 1953, 1952, 1951, 1950,
       1949, 1948, 1848, 1947, 1946, 1945, 1944, 1943, 1942, 1941, 1940,
       1939, 1938, 1937, 1936, 1935, 1934, 1933, 1932, 1931, 1930, 1929,
       1928, 1927, 1926, 1925, 1924, 1923, 1922, 1921, 1920, 1919, 1918,
       1917, 1916, 1915, 1914, 1913, 1912, 1911, 1910, 1909, 1908, 1907,
       1906, 1905, 1904, 1903, 1902, 1901, 1900, 1899, 1898, 1897, 1896,
       1895, 1894, 1893, 1892, 1891, 1890, 1889, 1888, 1887, 1886, 1885,
       1884, 1883, 1882, 1881, 1880, 1879, 1878, 18

# Cleaning the 'Date' Column

In this section, we'll handle the missing and inconsistent values in the 'Date' column to ensure the data is in the correct format for analysis.

In [23]:
# Identify and show the top 50 most common dates in the 'Date' column
# For quick overview of inconsistent formatting 
df['Date'].value_counts().head(50)

Date
1957                    9
1958                    7
1956                    6
1941                    6
No date                 6
28-Jul-1995             5
12-Apr-2001             5
2024-07-04 00:00:00     5
05-Oct-2003             5
1970s                   5
Aug-1956                5
1955                    5
1954                    5
1942                    5
No date, Before 1963    5
15-Apr-2018             4
1960                    4
1960s                   4
1959                    4
Before 1906             4
27-Dec-2008             4
29-Apr-2017             4
09-Jan-2010             4
14-Jun-2012             4
23-Jan-1970             4
27-Jul-1952             4
1952                    4
1950                    4
1949                    4
1945                    4
28-Dec-2014             4
04 Jul-2023             4
Reported 10-Oct-1906    4
09-Jul-1994             4
20-Sep-2015             4
Before 1958             4
02-Sep-2012             3
30-Sep-2007             3
11-Aug-

In [24]:
# Extract date patterns from the 'Date' column using regex
# The regex pattern looks for:
# - 4-digit years (e.g., 2023)
# - Numbers with ordinal suffixes (e.g., 1st, 2nd, 3rd, 4th)
# - Date ranges with hyphens (e.g., 1-2, 10-15)
# - Year ranges (e.g., 2020-2022)
# - Month names or other text (e.g., January, Feb)
date_patterns = df['Date'].astype(str).str.extract(r'(\d{4}|\d{1,2}(?:st|nd|rd|th)|\d{1,2}-\d{1,2}|\d{4}-\d{4}|[A-Za-z]+)')

# Display the 13 most common date patterns found in the dataset
print(f"\n THE MOST COMMON TEXT PATTERNS OF 'DATE' COLUMN ARE:\n", date_patterns[0].value_counts().head(13))


 THE MOST COMMON TEXT PATTERNS OF 'DATE' COLUMN ARE:
 0
Jul         657
Aug         591
Sep         514
Reported    512
Jan         494
Jun         471
Oct         438
Apr         435
Dec         430
Mar         415
May         400
Nov         396
Feb         378
Name: count, dtype: int64


In [25]:
# Inspect entries containing the word 'Reported'
# Filter the dataframe to only include rows where the 'Date' column contains the word "Reported" 
df_reported = df[df['Date'].astype(str).str.contains("Reported", case=False)]
print(df_reported['Date'].head(20))

118     Reported 02 Nov-2023
154     Reported 14 Jul-2023
163    Reported 14-June 2023
277     Reported 27-Apr-2022
298     Reported 10-Feb-2022
312     Reported 06-Dec-2021
351     Reported 11-Jul-2021
505     Reported 14-Mar-2020
510     Reported 30-Jan-2020
549     Reported 04-Oct-2019
577     Reported 17-Jul-2019
591     Reported 06-Jun-2019
606     Reported 27-Apr-2019
656     Reported 16-Oct-2018
674     Reported 28-Aug-2018
690     Reported 16-Jul-2018
697    Reported 09-Jul-2018.
719     Reported 30-Apr-2018
734     Reported 10-Apr-2018
764     Reported 25-Nov-2017
Name: Date, dtype: object


In [26]:
# Dictionary mapping various month name formats (including abbreviations and different languages) to standardized 3-letter format
month_mapping = {
    'jan': 'Jan', 'january': 'Jan',
    'feb': 'Feb', 'february': 'Feb', 'fev': 'Feb',
    'mar': 'Mar', 'march': 'Mar',
    'apr': 'Apr', 'april': 'Apr', 'abr': 'Apr',
    'may': 'May',
    'jun': 'Jun', 'june': 'Jun',
    'jul': 'Jul', 'july': 'Jul',
    'aug': 'Aug', 'august': 'Aug', 'ago': 'Aug',
    'sep': 'Sep', 'september': 'Sep', 'set': 'Sep',
    'oct': 'Oct', 'october': 'Oct', 'out': 'Oct',
    'nov': 'Nov', 'november': 'Nov',
    'dec': 'Dec', 'december': 'Dec', 'dez': 'Dec'
}

## Create a regex pattern by joining all month keys with '|' (OR operator)
all_month_patterns = '|'.join(month_mapping.keys())

# Extract month from 'Date' column using regex pattern:
# 1. Convert Date column to string
# 2. Extract month name using case-insensitive regex (?i)
# 3. Convert extracted month to lowercase
# 4. Map to standardized format using the month_mapping dictionary
df['Month'] = df['Date'].astype(str).str.extract(f'(?i)({all_month_patterns})', expand=False).str.lower().map(month_mapping)

In [27]:
# Analyse the new column 'Month' to understand if the value counts coincide with the date-paterns
df['Month'].value_counts()

Month
Jul    753
Aug    652
Sep    569
Jan    546
Jun    527
Oct    481
Apr    477
Dec    475
Mar    446
May    434
Nov    430
Feb    411
Name: count, dtype: int64

In [28]:
# Inspect the newly created 'Month' column
df.Month.info()

<class 'pandas.core.series.Series'>
Index: 6757 entries, 0 to 7041
Series name: Month
Non-Null Count  Dtype 
--------------  ----- 
6201 non-null   object
dtypes: object(1)
memory usage: 363.6+ KB


In [29]:
# Inspect all null entries in the 'Month' column 
# df[df['Month'].isna()] 

#Inspect null values in the 'Month' column compared to the values in 'Date' column
# print(df[df['Month'].isnull()][['Date', 'Month']].head(20))
# print(df[df['Month'].isnull()][['Date', 'Month']].tail(20))
df[df['Month'].isnull()][['Date', 'Month']].sample(30)

,Date,Month
5149,1952,NaN
6909,Ca 1200-1500 A.D.,NaN
6759,1850,NaN
6914,Ca. 336.B.C..,NaN
6966,"No date, Before 1975",NaN
32,2025-02-27 00:00:00,NaN
5413,1943,NaN
6927,Before 1934,NaN
5081,1954,NaN
6954,1950s,NaN


In [30]:
# Create a boolean mask to filter rows where 'Date' column matches the format 'YYYY-MM-DD HH:MM:SS'
mask = df['Date'].astype(str).str.match(r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$')

# Filter the dataframe to keep only rows that match the date format pattern
# and select only the 'Date' and 'Month' columns
df_filtered = df.loc[mask, ['Date', 'Month']]

# Display the first 20 rows of the filtered dataframe
print(df_filtered.head(20)) 

# Print the total number of rows that match the date format pattern
print(f"Total found: {len(df_filtered)}")

                   Date Month
19  2025-06-11 00:00:00   NaN
21  2025-05-29 00:00:00   NaN
22  2025-05-26 00:00:00   NaN
23  2025-05-15 00:00:00   NaN
24  2025-05-08 00:00:00   NaN
25  2025-04-21 00:00:00   NaN
26  2025-04-20 00:00:00   NaN
27  2025-04-19 00:00:00   NaN
28  2025-04-12 00:00:00   NaN
29  2025-03-26 00:00:00   NaN
30  2025-03-10 00:00:00   NaN
31  2025-03-07 00:00:00   NaN
32  2025-02-27 00:00:00   NaN
34  2025-02-22 00:00:00   NaN
35  2025-02-22 00:00:00   NaN
36  2025-02-22 00:00:00   NaN
37  2025-02-10 00:00:00   NaN
38  2025-02-10 00:00:00   NaN
40  2025-02-03 00:00:00   NaN
41  2025-01-23 00:00:00   NaN
Total found: 67


In [31]:
# Dictionary mapping numeric month representations to abbreviated month names
numeric_month_mapping = {
    '01': 'Jan', '02': 'Feb', '03': 'Mar', '04': 'Apr', '05': 'May', '06': 'Jun',
    '07': 'Jul', '08': 'Aug', '09': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec'
}

# Create a boolean mask for rows where 'Month' column contains NaN values
nan_mask = df['Month'].isna()

# For rows with missing month values, extract the month part (MM) from the 'Date' column
# using regex pattern that matches YYYY-MM-DD format
extracted_numeric_months = df.loc[nan_mask, 'Date'].astype(str).str.extract(r'\d{4}-(\d{2})-\d{2}', expand=False)

# Replace NaN values in 'Month' column with the corresponding month abbreviations
df.loc[nan_mask, 'Month'] = extracted_numeric_months.map(numeric_month_mapping)

In [32]:
# Reinspect the 'Month' to confirm extraction of values formatted as 'YYYY-MM-DD HH:MM:SS' in the 'Date' column
print(df['Month'].value_counts(), '\n')
print(df['Month'].info(), '\n')
print(df['Month'].unique())

Month
Jul    763
Aug    654
Sep    572
Jan    552
Jun    534
Oct    485
Apr    485
Dec    481
Mar    449
May    442
Nov    433
Feb    418
Name: count, dtype: int64 

<class 'pandas.core.series.Series'>
Index: 6757 entries, 0 to 7041
Series name: Month
Non-Null Count  Dtype 
--------------  ----- 
6268 non-null   object
dtypes: object(1)
memory usage: 363.6+ KB
None 

['Aug' 'Jul' 'Jun' 'May' 'Apr' 'Mar' 'Feb' 'Jan' 'Dec' 'Nov' 'Oct' 'Sep'
 nan]


In [33]:
# Relocate 'Month' column at the second position (index 1) 
cols = list(df.columns)
cols.remove('Month')
cols.insert(1, 'Month')
df = df[cols]